In [1]:
import cv2 as cv
print(cv.__version__)

4.5.1


### numpy array indexing

In [2]:
img_test = cv.imread('test.jpg')
print(type(img_test))
print(img_test.shape)
print(img_test[90:,90:,0]) 

print()

img_cvtGray = cv.cvtColor(img_test, cv.COLOR_BGR2GRAY)
print(img_cvtGray[90:,90:])
print(img_cvtGray.shape)

<class 'numpy.ndarray'>
(323, 430, 3)
[[147 167 214 ... 237 237 237]
 [169 219 226 ... 237 237 237]
 [216 225 222 ... 238 238 238]
 ...
 [240 240 240 ... 239 239 239]
 [240 240 240 ... 239 239 239]
 [240 240 240 ... 239 239 239]]

[[158 174 218 ... 237 237 237]
 [176 223 228 ... 237 237 237]
 [220 227 224 ... 238 238 238]
 ...
 [240 240 240 ... 239 239 239]
 [240 240 240 ... 239 239 239]
 [240 240 240 ... 239 239 239]]
(323, 430)


### 얕은 복사(같은 id) vs 깊은 복사(다른 id)

In [3]:
# 1. 얕은 복사
a = [1,2,3]
b = a
print(id(a), id(b)) # 같은 id (같은 주소값)

b[0] = 5000
print(a, b) # 같은 곳을 가리키므로, b를 바꿔도 a값까지 바뀜!

print()

# 2. 깊은 복사
c = [1,2,3]
d = c.copy()
print(id(c), id(d))

d[0] = 5000
print(c, d)

2741612483968 2741612483968
[5000, 2, 3] [5000, 2, 3]

2741612482816 2741612484544
[1, 2, 3] [5000, 2, 3]


### 함수로 배열 넘기기
함수로 배열을 넘기는 행위가 배열 자체를 전달해서 새로운 배열로 복사하는 것이 아니라, **"주소값"**을 넘겨주는 것이다!

In [4]:
import numpy as np

def test(arr):
    print(id(arr))

arr_test = np.ones(10)
print(id(arr_test))
test(arr_test)    

2741633105392
2741633105392


### cv2.equalizeHist(ds.pixel_array)
The function equalizeHist is histogram equalization of images and only implemented for CV_8UC1 type, which is a single channel 8 bit unsigned integral type.  
> 따라서 원본이 1000 넘는 값들로 이루어져 uint8 (8bit, 256)이 아니므로, 이 opencv함수 말고 직접 코딩해야됨...  
>  ==> **map_uint16_to_uint8() 함수!**

### cf. fail modules
* ArrayDicom = np.zeros(shape, dtype=np.uint8)
* cv.equalizeHist()
* cv.createCLAHE()
* cv.normalize()
* skimage.util.img_as_ubyte() (from skimage import ima_as_ubyte) 

<hr>

In [5]:
import cv2 as cv
import glob
import pydicom as dicom
import os
import numpy as np

# array indexing using np.where
def Overlay(back, front, rows, cols):
    back[:, :, 0] = np.where(front != 0, back[:, :, 0] * 0.7 + 255 * 0, 0)
    back[:, :, 1] = np.where(front != 0, back[:, :, 1] * 0.7 + 255 * 0, 0)
    back[:, :, 2] = np.where(front != 0, back[:, :, 2] * 0.7 + 255 * 0.3, 0)
    
    #print(back.shape) # (512, 512, 3) ==> (열, 행, 면)
    
    '''
    for i in range(rows):
        for j in range(cols):
            if (front[i][j] == 0) : continue
            
            # linear interpolation
            back[i][j][0] = back[i][j][0] * 0.7 + 255 * 0 # b
            back[i][j][1] = back[i][j][1] * 0.7 + 255 * 0 # g
            back[i][j][2] = back[i][j][2] * 0.7 + 255 * 0.3 # r
    '''

# from 0 ~ 4095 (16bit, uint16) to 0 ~ 255 (8bit, uint8)
def map_uint16_to_uint8(img, lower_bound=None, upper_bound=None):
    lower_bound = np.min(img)
    upper_bound = np.max(img)
    
    if not(0 <= lower_bound < 2**16) and lower_bound is not None:
        raise ValueError(
            '"lower_bound" must be in the range [0, 65535]')
    if not(0 <= upper_bound < 2**16) and upper_bound is not None:
        raise ValueError(
            '"upper_bound" must be in the range [0, 65535]')
    if lower_bound is None:
        lower_bound = np.min(img)
    if upper_bound is None:
        upper_bound = np.max(img)
    if lower_bound >= upper_bound:
        raise ValueError(
            '"lower_bound" must be smaller than "upper_bound"')
    lut = np.concatenate([
        np.zeros(lower_bound, dtype=np.uint16),
        np.linspace(0, 255, upper_bound - lower_bound).astype(np.uint16),
        np.ones(2**16 - upper_bound, dtype=np.uint16) * 255
    ])
    return lut[img].astype(np.uint8)



def main():
    files_str = [file for file in glob.glob("C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT/*.dcm")] # imrepyad(dcm) : None
    
    for f in files_str:
        print(f)
        ori = dicom.read_file(f) # get dicom file / type : <class 'pydicom.dataset.FileDataset'> / size : (512, 512)       
        rows, cols = ori.Rows, ori.Columns
        
        ori1 = map_uint16_to_uint8(ori.pixel_array) # type : np.ndarray ==> cv.imread img도 np.ndarray형!
        img_copy = ori1.copy()
        
        
        # 1. Bilateral Filtering (noise filtering)
        filtered = cv.bilateralFilter(img_copy, -1, 15, 15)
        
        
        # 2. Otsu;s Thresholding (cv2.threshold(src, 0(**T not defined**), 255(change to 255), flag(binary & otsu)))
        ret,otsu = cv.threshold(filtered, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU) # 히스토그램 분석 후 자동으로 T값(ret) 구해서 적용
        
        
        # 3. Morphology_preprocessing(remove outlines with erode) ==> anchor(point) default : (-1, -1)
        mask = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
        pre_erode = cv.erode(otsu, mask, iterations = 7)
        pre_dilate = cv.dilate(pre_erode, mask, iterations = 7)
        
        
        # 4. Floodfill (combine background to select hole in body) ==> mask : None (not used)
        hole = pre_dilate.copy()
        cv.floodFill(hole, None, (0, 0), 255)
        cv.floodFill(hole, None, (cols-1, rows-1), 255)
    
    
        # 5. Invert hole
        hole_inv = cv.bitwise_not(hole) # == np.bitwise_not == np.invert
        
        
        # 6. bitwise OR (combine pre(bone) and hole)
        bitor = np.bitwise_or(pre_dilate, hole_inv)
        
        
        # 7. overlay
        back3C = cv.cvtColor(ori1, cv.COLOR_GRAY2BGR) # 3channel
        front = bitor.copy()
        Overlay(back3C, front, rows, cols)
        
        
        result_path = "C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT/result_python/"
        cv.imwrite(result_path + f[-10:-4] + '.png', back3C) # f : "~/CT0002.dcm"
        
    
    
main()

C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0002.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0003.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0004.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0005.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0006.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0007.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0008.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0009.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0010.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0011.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0012.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0

C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0101.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0102.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0103.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0104.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0105.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0106.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0107.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0108.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0109.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0110.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0111.dcm
C:/Users/Administrator/Desktop/jiwon/dilab/Projects/180509_SampleData_CT\CT0

<hr>

## [PyQt]

### select file

In [3]:
import sys
from PyQt5.QtWidgets import *

In [3]:
# prevent kernel from dying
from PyQt5.QtWidgets import QApplication, QDialog
from PyQt5.QtCore import QCoreApplication

In [3]:
class MyWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setupUI()

    def setupUI(self):
        self.setGeometry(800, 200, 300, 300)
        self.setWindowTitle("Get Files")

        self.pushButton = QPushButton("Select Folder")
        self.pushButton.clicked.connect(self.pushButtonClicked)
        self.label = QLabel()

        layout = QVBoxLayout()
        layout.addWidget(self.pushButton)
        layout.addWidget(self.label)

        self.setLayout(layout)

    def pushButtonClicked(self):
        '''
        fname = QFileDialog.getOpenFileName(self) # 해당 파일의 절대 경로 반환 (tuple)
        self.label.setText(fname[0]) # tuple의 0번째 값(경로) 창에 출력
        print(fname)
        '''
        options = QFileDialog.Options()
        options |= QFileDialog.ShowDirsOnly
        working_path = QFileDialog.getExistingDirectory(self, "select Directory")
        return working_path

def getPath():
    # prevent kernel from dying
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
        
    window = MyWindow()
    window.show()
    src_path = window.pushButtonClicked()
    #app.exec_() => ?
    return src_path

### drag and drop

In [8]:
from PyQt5.QtWidgets import QMainWindow, QApplication
import sys


class MainWidget(QMainWindow): # QMainWindow class를 상속하여 MainWidget class 생성
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Drag and Drop")
        self.resize(720, 480)
        self.setAcceptDrops(True) # enable drop events for the widget
        self.files = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls(): # if data has urls, accept the event
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event): # drop event 발생 시 수행할 것 정의
        self.files = [u.toLocalFile() for u in event.mimeData().urls()]
        '''
        for f in self.files:
            print('f in self.files :',f)
        '''

        

def dragAndDrop():
    app = QApplication(sys.argv) # QT App 객체 생성
    ui = MainWidget()
    ui.show()
    
    # app.exec_() : execute. 프로그램이 꺼지지 않고 '무한 루프' 상태로 만들기. 앱 종료시 0 return.
    # sys.exit(0) : 프로그램 종료로 받아들이고 실행중인 코드 중단시킴
    app.exec_() 

    
    path = ui.files # app.exec_()전에 ui.files 접근 x!! 아직 프로그램 실행 전!
    
    return path

In [10]:
print('dragAndDrop() return path :', dragAndDrop())

f in self.files : D:/_jiwon_/datasets
dragAndDrop() return path : ['D:/_jiwon_/datasets']


### sub dir search

In [13]:
import os

def search(dirname):
    print('dirname :', dirname)
    
    # try~except : os.listdir()시 권한 없는 dir에 접근하더라도 오류로 종료되는 것 방지
    try:
        filenames = os.listdir(dirname) # os.listdir(dir) : 해당 dir안의 파일들 list 구하기(only 파일명)
        print('num :', len(filenames))
        
        for filename in filenames:
            full_filename = os.path.join(dirname, filename) # dirname + filename (full path)
            print(full_filename)
            
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                ext = os.path.splitext(full_filename)[-1] # ext : 확장자. 확장자 기준으로 두 부분으로 나눔
                '''
                if ext == '.dcm':
                    print(full_filename)
                '''
    except PermissionError:
        pass
    

dirname = dragAndDrop()
search(dirname[0])

f in self.files : D:/_jiwon_/datasets
dirname : D:/_jiwon_/datasets
folder num : 16
D:/_jiwon_/datasets\180509_SampleData_CT
dirname : D:/_jiwon_/datasets\180509_SampleData_CT
folder num : 193
D:/_jiwon_/datasets\180509_SampleData_CT\CT0002.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0003.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0004.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0005.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0006.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0007.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0008.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0009.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0010.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0011.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0012.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0013.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0014.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0015.dcm
D:/_jiwon_/datasets\180509_SampleData_CT\CT0016.dcm
D:/_jiwon_/datasets\180509_

D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0057.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0058.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0059.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0060.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0061.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0062.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0063.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0064.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0065.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0066.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0067.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0068.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0069.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0070.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0071.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0072.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0073.dcm
D:/_jiwon_/datasets\190212_Patient3b_CT\Breast0074.dcm
D:/_jiwon_

D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0182.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0183.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0184.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0185.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0186.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0187.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0188.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0189.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0190.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0191.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0192.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0193.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0194.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0195.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0196.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0197.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0198.dcm
D:/_jiwon_/datasets\190426_Patient6b(6)_CT\FILE0

folder num : 161
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0002.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0003.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0004.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0005.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0006.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0007.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0008.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0009.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0010.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0011.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0012.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0013.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0014.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0015.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0016.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0017.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0018.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0019.dcm
D:/_jiwon_/datasets\190531_Patient9_CT\FILE0020

D:/_jiwon_/datasets\191004_Patient11_CT\FILE0029.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0030.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0031.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0032.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0033.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0034.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0035.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0036.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0037.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0038.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0039.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0040.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0041.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0042.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0043.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0044.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0045.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE0046.dcm
D:/_jiwon_/datasets\191004_Patient11_CT\FILE00

D:/_jiwon_/datasets\200206_Patient13_CT\10021.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10022.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10023.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10024.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10025.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10026.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10027.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10028.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10029.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10030.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10031.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10032.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10033.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10034.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10035.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10036.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10037.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10038.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10039.dcm
D:/_jiwon_/datasets\200206_Patient13_CT\10040.dcm


### apply to CTSS

In [4]:
import cv2 as cv
import glob
import pydicom as dicom
import os
import numpy as np

# array indexing using np.where
def Overlay(back, front, rows, cols):
    back[:, :, 0] = np.where(front != 0, back[:, :, 0] * 0.7 + 255 * 0, 0)
    back[:, :, 1] = np.where(front != 0, back[:, :, 1] * 0.7 + 255 * 0, 0)
    back[:, :, 2] = np.where(front != 0, back[:, :, 2] * 0.7 + 255 * 0.3, 0)
    
    #print(back.shape) # (512, 512, 3) ==> (열, 행, 면)
    
    '''
    for i in range(rows):
        for j in range(cols):
            if (front[i][j] == 0) : continue
            
            # linear interpolation
            back[i][j][0] = back[i][j][0] * 0.7 + 255 * 0 # b
            back[i][j][1] = back[i][j][1] * 0.7 + 255 * 0 # g
            back[i][j][2] = back[i][j][2] * 0.7 + 255 * 0.3 # r
    '''

# from 0 ~ 4095 (16bit, uint16) to 0 ~ 255 (8bit, uint8)
def map_uint16_to_uint8(img, lower_bound=None, upper_bound=None):
    lower_bound = np.min(img)
    upper_bound = np.max(img)
    
    if not(0 <= lower_bound < 2**16) and lower_bound is not None:
        raise ValueError(
            '"lower_bound" must be in the range [0, 65535]')
    if not(0 <= upper_bound < 2**16) and upper_bound is not None:
        raise ValueError(
            '"upper_bound" must be in the range [0, 65535]')
    if lower_bound is None:
        lower_bound = np.min(img)
    if upper_bound is None:
        upper_bound = np.max(img)
    if lower_bound >= upper_bound:
        raise ValueError(
            '"lower_bound" must be smaller than "upper_bound"')
    lut = np.concatenate([
        np.zeros(lower_bound, dtype=np.uint16),
        np.linspace(0, 255, upper_bound - lower_bound).astype(np.uint16),
        np.ones(2**16 - upper_bound, dtype=np.uint16) * 255
    ])
    return lut[img].astype(np.uint8)



def main():
    src_path = getPath()
    print('src dir path :', src_path)
    files_str = [file for file in glob.glob(src_path+"/*.dcm")] # imrepyad(dcm) : None
    
    for f in files_str:
        print(f)
        ori = dicom.read_file(f) # get dicom file / type : <class 'pydicom.dataset.FileDataset'> / size : (512, 512)       
        rows, cols = ori.Rows, ori.Columns
        
        ori1 = map_uint16_to_uint8(ori.pixel_array) # type : np.ndarray ==> cv.imread img도 np.ndarray형!
        img_copy = ori1.copy()
        
        
        # 1. Bilateral Filtering (noise filtering)
        filtered = cv.bilateralFilter(img_copy, -1, 15, 15)
        
        
        # 2. Otsu;s Thresholding (cv2.threshold(src, 0(**T not defined**), 255(change to 255), flag(binary & otsu)))
        ret,otsu = cv.threshold(filtered, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU) # 히스토그램 분석 후 자동으로 T값(ret) 구해서 적용
        
        
        # 3. Morphology_preprocessing(remove outlines with erode) ==> anchor(point) default : (-1, -1)
        mask = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
        pre_erode = cv.erode(otsu, mask, iterations = 7)
        pre_dilate = cv.dilate(pre_erode, mask, iterations = 7)
        
        
        # 4. Floodfill (combine background to select hole in body) ==> mask : None (not used)
        hole = pre_dilate.copy()
        cv.floodFill(hole, None, (0, 0), 255)
        cv.floodFill(hole, None, (cols-1, rows-1), 255)
    
    
        # 5. Invert hole
        hole_inv = cv.bitwise_not(hole) # == np.bitwise_not == np.invert
        
        
        # 6. bitwise OR (combine pre(bone) and hole)
        bitor = np.bitwise_or(pre_dilate, hole_inv)
        
        
        # 7. overlay
        back3C = cv.cvtColor(ori1, cv.COLOR_GRAY2BGR) # 3channel
        front = bitor.copy()
        Overlay(back3C, front, rows, cols)
        
        # src_path : D:/_jiwon_/datasets/180509_SampleData_CT
        result_path = src_path + '/result/'
        cv.imwrite(result_path + f[-10:-4] + '.png', back3C) # f : "~/CT0002.dcm" => **슬라이싱 수정하기!**
        
    
    
main()

src dir path : D:/_jiwon_/datasets/180509_SampleData_CT
D:/_jiwon_/datasets/180509_SampleData_CT\CT0002.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0003.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0004.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0005.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0006.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0007.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0008.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0009.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0010.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0011.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0012.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0013.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0014.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0015.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0016.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0017.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0018.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0019.dcm
D:/_jiwo

D:/_jiwon_/datasets/180509_SampleData_CT\CT0159.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0160.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0161.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0162.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0163.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0164.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0165.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0166.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0167.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0168.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0169.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0170.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0171.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0172.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0173.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0174.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0175.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0176.dcm
D:/_jiwon_/datasets/180509_SampleData_CT\CT0177.dcm
D:/_jiwon_/d